# CREATE GEOMETRY and FLUXES FROM DATA

## Import

In [1]:
# A 
import ast
# C
from collections import defaultdict
# G
import gc
import geopandas as gpd
# J
import json
# M
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from multiprocessing import Pool
# N
from numba import prange
import numpy as np
# O
import os
# P
import pandas as pd
# S
from shapely.geometry import box,LineString,Point,MultiPoint,MultiLineString,MultiPolygon,Polygon
from shapely.ops import unary_union
import socket
import sys
# T
from termcolor import  cprint
import time

# Project specific
# A
from AlgorithmCheck import *
# C
from ComputeGrid import *
from ComputeHexagon import *
# F
current_dir = os.path.join(os.getcwd()) 
mother_path = os.path.abspath(os.path.join(current_dir, os.pardir))
print('mother_path:', mother_path)
sys.path.append(os.path.join(mother_path, 'PreProcessing'))
sys.path.append(os.path.join(mother_path))
from FittingProcedures import *
# G
from GeometrySphere import *
from GenerateModifiedFluxesSimulation import *
from GravitationalFluxes import *                                               # FIT section
from Grid import *
# H 
from Hexagon import *
if socket.gethostname()=='artemis.ist.berkeley.edu':
    sys.path.append(os.path.join('/home/alberto/LPSim','traffic_phase_transition','scripts','ServerCommunication'))
else:
    sys.path.append(os.path.join(os.getenv('TRAFFIC_DIR'),'scripts','ServerCommunication'))
from HostConnection import *
# M
from MainPolycentrism import *
from ModifyPotential import *
# O 
from ODfromfma import *
# P
from plot import *
from Polycentrism import *
from PolycentrismPlot import *
from PolygonSettings import *
from Potential import *
from PreprocessingObj import *



## BASIC PARAMS
gc.set_threshold(10000,50,50)
plt.rcParams.update({
    "text.usetex": False,
})
StateAlgorithm = InitWholeProcessStateFunctions()



mother_path: /home/alberto/LPSim/traffic_phase_transition/scripts
PyMC3 not installed


## Single City Variables

In [2]:
# Always Needed
NameCity = "BOS"
NameCities = ["BOS"]

grid_size = 0.02
hexagon_resolution = 8
# Computation Grid
ArgsComputationGrid = [(NameCity,TRAFFIC_DIR,[grid_size])]
ArgsComputationHexagon = [(NameCity,TRAFFIC_DIR,[hexagon_resolution])]

## Multiple Cities Variables

In [ ]:
# Always Needed
NameCities = os.listdir(os.path.join(TRAFFIC_DIR,'data','carto'))
if len(NameCities)==0:
    cprint('No city found in the data/cart folder', 'red')
    sys.exit()
# Computation Grid
ArgsComputationGrid = [(NameCities[i],TRAFFIC_DIR,[grid_size]) for i in range(len(list_cities))]
ArgsComputationHexagon = [(NameCities[i],TRAFFIC_DIR,[hexagon_resolution]) for i in range(len(list_cities))]
grid_size = 0.02
hexagon_resolution = 8

### Compute Grid after Hexagon

In [ ]:
for arg in ArgsComputationHexagon:
    ComputeHexagon(*arg)
for arg in ArgsComputationGrid:
    GeometricalInfo = GeometricalSettingsSpatialPartition(NameCity,TRAFFIC_DIR)
    ComputeGrid(GeometricalInfo,*arg)


# COMPUTE VECTOR FIELD

### Description: Upload grid that must be already computed,
####

In [4]:
for NameCity in NameCities: 
    dir_grid = os.path.join(TRAFFIC_DIR,'data','carto',NameCity,'grid',str(grid_size))
    ## BASIC NEEDED OBJECTS
    if not StateAlgorithm["GeometricalSettingsSpatialPartition"]:
        SFO_obj = GeometricalSettingsSpatialPartition(NameCity,TRAFFIC_DIR)
        bbox = SFO_obj.bounding_box
        minx, miny, maxx, maxy = bbox
    else:
        pass
    DirGeometry = os.path.join(TRAFFIC_DIR,'data','carto','{0}'.format(NameCity))
    if not StateAlgorithm["GetGrid"]:
        grid = GetGrid(grid_size,SFO_obj.bounding_box,'EPSG:4326',DirGeometry)
        StateAlgorithm["GetGrid"] = True
    else:
        pass
    if not StateAlgorithm["GetBoundariesInterior"]:
        grid = GetBoundariesInterior(grid,SFO_obj)
        StateAlgorithm["GetBoundariesInterior"] = True
    else:
        pass
    if not StateAlgorithm["GetDirectionMatrix"]:
        df_distance,FoundDistMat = GetDirectionMatrix(DirGeometry,grid_size)
        StateAlgorithm["GetDirectionMatrix"] = True
    else:
        pass
    if not StateAlgorithm["GetODGrid"]:
        Tij = GetODGrid(DirGeometry,str(grid_size))
        StateAlgorithm["GetODGrid"] = True
    else:
        pass
    if not StateAlgorithm["GetLattice"]:
        lattice = nx.read_graphml(os.path.join(dir_grid,"centroid_lattice.graphml"))
        StateAlgorithm["GetLattice"] = True
    else:
        pass
    if not StateAlgorithm["GetVectorField"]:
        VectorField = GetVectorField(Tij,df_distance)
        StateAlgorithm["GetVectorField"] = True
    VectorFieldDir = os.path.join(TRAFFIC_DIR,'data','carto',NameCity,'grid',str(grid_size))
    if not StateAlgorithm["GetPotentialLattice"]:
        lattice = GetPotentialLattice(lattice,VectorField)
        StateAlgorithm["GetPotentialLattice"] = True
    else:
        pass
    if not StateAlgorithm["GetPotentialDataframe"]:
        lattice = SmoothPotential(lattice)
        StateAlgorithm["GetPotentialDataframe"] = True
    else:
        pass
    if not StateAlgorithm["ConvertLattice2PotentialDataframe"]:
        PotentialDataframe = ConvertLattice2PotentialDataframe(lattice)
        StateAlgorithm["ConvertLattice2PotentialDataframe"] = True
    else:
        pass
    if not StateAlgorithm["CompletePotentialDataFrame"]:
        PotentialDataframe = CompletePotentialDataFrame(VectorField,grid,PotentialDataframe)
        StateAlgorithm["CompletePotentialDataFrame"] = True
    else:
        pass
    if os.path.isfile(os.path.join(dir_grid,'PotentialDataframe.csv')):
        SavePotentialDataframe(PotentialDataframe,dir_grid)
    if os.path.isfile(os.path.join(VectorFieldDir,'VectorField.csv')):
        SaveVectorField(VectorField,VectorFieldDir)
    SumPot = np.sum(PotentialDataframe['V_out']) 
    NumGridEdge = grid[grid['relation_to_line']=='edge'].shape[0]




# FIT GRAVITATIONAL MODEL

In [ ]:
"""
    Compute The Fit From Data.
"""
if not StateAlgorithm["GeometricalSettingsSpatialPartition"]:
    SFO_obj = GeometricalSettingsSpatialPartition(NameCity,TRAFFIC_DIR)
    bbox = SFO_obj.bounding_box
    minx, miny, maxx, maxy = bbox
    DirGeometry = os.path.join(TRAFFIC_DIR,'data','carto','{0}'.format(NameCity))
if not StateAlgorithm["GetGrid"]:
    grid = GetGrid(grid_size,SFO_obj.bounding_box,'EPSG:4326',DirGeometry)
    StateAlgorithm["GetGrid"] = True
else:
    pass
if not StateAlgorithm["GetBoundariesInterior"]:
    grid = GetBoundariesInterior(grid,SFO_obj)
    StateAlgorithm["GetBoundariesInterior"] = True
else:
    pass
if not StateAlgorithm["GetDirectionMatrix"]:
    df_distance,FoundDistMat = GetDirectionMatrix(DirGeometry,grid_size)
    StateAlgorithm["GetDirectionMatrix"] = True
else:
    pass
if not StateAlgorithm["GetODGrid"]:
    Tij = GetODGrid(DirGeometry,str(grid_size))
    StateAlgorithm["GetODGrid"] = True
else:
    pass

for NameCity in NameCities:
    potential_dir = os.path.join(TRAFFIC_DIR,'data','carto',NameCity,'grid',str(grid_size),"potential")
    VespignaniBlock(df_distance,grid,Tij,potential_dir)




# MODIFY POTENTIAL

In [ ]:
for NameCity in NameCities:
    with open(os.path.join(TRAFFIC_DIR,'data','carto',NameCity,'grid','0.02','origindest2grid.json'),'r') as f:
        OD2grid = json.load(f)
    with open(os.path.join(TRAFFIC_DIR,'data','carto',NameCity,'grid','0.02','grid2origindest.json'),'r') as f:
        grid2OD = json.load(f)
    with open(os.path.join(TRAFFIC_DIR,'data','carto',NameCity,'osmid2idx.json'),'r') as f:
        osmid2index = json.load(f)

    list_peaks = [2,3,4,5,6,8,10,100,200,500] # [2,3,4,5,6,8,10,
    InfoConfigurationPolicentricity = {num_peaks: defaultdict() for num_peaks in list_peaks}
    Covarinaces = [1,2,4,8,10,15,20,30,50,100]
    distributions = ['exponential']   
    parallel = False
    if not parallel:  
        for cv in Covarinaces:
            for distribution in distributions:#['gaussian','exponential']:#,'exponential']: # NOTE: There is some problem with the exponential distribution
                InfoCenters = {'center_settings': {"type":distribution},'covariance_settings':{"covariances":{"cvx":cv,"cvy":cv},"Isotropic": True,"Random": False}}
                #parameter_values = ((InfoConfigurationPolicentricity,grid,SFO_obj,Tij,df_distance,num_peaks) for num_peaks in list_peaks)
                for num_peaks in list_peaks:
                    InfoConfigurationPolicentricity,UCI = ModifyMorphologyCity(InfoConfigurationPolicentricity,grid,SFO_obj,Tij,df_distance,lattice,num_peaks,TRAFFIC_DIR,NameCity,grid_size,InfoCenters,fraction_fluxes = 200,verbose = True)
                    if socket.gethostname()=='artemis.ist.berkeley.edu':
                        SaveOd = "/home/alberto/LPSim/LivingCity/berkeley_2018/new_full_network"
                    else:
                        SaveOd = f'/home/aamad/Desktop/phd/traffic_phase_transition/data/carto/{NameCity}/OD'
                    OD_from_T_Modified(InfoConfigurationPolicentricity[num_peaks]['Tij'],
                                    CityName2RminRmax,
                                    NameCity,
                                    osmid2index,
                                    grid2OD,
                                    1,
                                    SaveOd,
                                    7,
                                    8,
                                    round(UCI,3))
    else:
        Args_OD_from_T_Modified = [(num_peaks, cv, distribution,InfoConfigurationPolicentricity,grid,SFO_obj,Tij,df_distance,lattice,TRAFFIC_DIR,NameCity,grid_size) for distribution in distributions for num_peaks in list_peaks for cv in Covarinaces]
        with Pool(10) as p:
            p.starmap(GenerateParallelODs, Args_OD_from_T_Modified)



process ComputeNewFluxConfiguration{
    input
}